In [1]:
include("../../src/WaterLily.jl")

using Plots
using Statistics

In [2]:
optPois = "Pois"
optSour = "Cylinder"
optBCon = "Per"

stepbystep = false

computationID = optPois * "_" * optSour * "_" * optBCon * "_"
print(computationID,"\n")

Pois_Cylinder_Neu_


In [3]:
T=Float32
N = (5*2^7,2^7)
#N = (2^8,2^8)
D = length(N)
Ng = N .+ 2
Nd = (Ng..., 2)
figureSize = Int.(N.*3.75)

l = N[2]
xc, yc = N[1]/2+1.5,N[2]/2+1.5
R = 2^5
U=1

optSour=="Cylinder" && function SourceFunc(x,y)
    r = sqrt.((x.-xc).^2+(y.-yc).^2)

    return -32*pi^6*R^4*U^2/l^6*(
        (cosh.(2*pi*(x.-xc)/l).+cos.(2*pi*(y.-yc)/l))./
        (cosh.(2*pi*(x.-xc)/l).-cos.(2*pi*(y.-yc)/l)).^3
    ).*(r .> 0.99R)
end

optSour=="Cylinder" && function ManufacSol(x,y)
    r = sqrt.((x.-xc).^2+(y.-yc).^2)

    return 2*R^2*pi^2*U^2/l^4*(
        (l^2*cosh.(2*pi*(x.-xc)/l).*cos.(2*pi*(y.-yc)/l).-pi^2*R^2 .-l^2)./
        (cosh.(2*pi*(x.-xc)/l).-cos.(2*pi*(y.-yc)/l)).^2
    ).*(r .> 0.99R)
end;

optSour=="Cylinder" && function UGradU(i, xyz)
    x,y = @. (xyz.- Array([xc,yc]))*2*pi/l
    coeff = 8*U^2*R^2*pi^3/l^5 ./ (-cosh.(x).+cos.(y)).^3
    commonInner = pi^2*R^2+l^2 .- l^2*cos(y).*cosh(x)/2
    i==1 && return coeff*(commonInner .- l^2* cos(y).^2/2).*sinh(x) # u_x
    i==2 && return coeff*(commonInner .- l^2*cosh(x).^2/2).* sin(y) # u_y
    return 0.                              # u_z
end;

optSour=="OneCylinder" && function SourceFunc(x,y)
    r = sqrt.((x.-xc).^2+(y.-yc).^2)

    return -8*U^2*R^4 ./r.^6 .*(r .> 0.99R)
end

optSour=="OneCylinder" && function ManufacSol(x,y)
    r = sqrt.((x.-xc).^2+(y.-yc).^2)

    return -U^2*R^2/2*(R^2 .-2*(x.-xc).^2 .+2*(y.-yc).^2)./r.^4 .*(r .> 0.99R)
end;

In [4]:
body = WaterLily.AutoBody((x,t)->√sum(abs2, x .- Array([xc,yc])) - R);
flow = WaterLily.Flow(N,(0,0));
flow.μ₀ .= 1.0;
WaterLily.measure!(flow,body;ϵ=1);
μ₀ = flow.μ₀;
#WaterLily.BC!(μ₀,(1,1))

642×130×2 Array{Float64, 3}:
[:, :, 1] =
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 ⋮                        ⋮              ⋱            ⋮                   
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  

In [5]:
#optPois=="MulP" && WaterLily.BC!(μ₀,ntuple(zero, D))

X = transpose(reshape([i-0.5 for i=1:Ng[1] for j=1:Ng[2]],Ng[2],Ng[1]))
Y = transpose(reshape([j-0.5 for i=1:Ng[1] for j=1:Ng[2]],Ng[2],Ng[1]))

source = SourceFunc(X,Y);
realSol = ManufacSol(X,Y);
originalSource = copy(source);
solIni = zeros(size(realSol));

# BCPer!(solIni)
# WaterLily.BCPer!(source)
# WaterLily.BC(source)
# BCPerVec!(μ₀)

642×130 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 ⋮                        ⋮              ⋱            ⋮                   
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0   

In [6]:
optPois=="Pois" && (pp = WaterLily.Poisson(solIni, μ₀, source));
optPois=="MulP" && (pp = WaterLily.MultiLevelPoisson(solIni, μ₀, source));

In [7]:
reconstructedSource = zeros(size(source))

for i∈2:N[1]+1, j∈2:N[2]+1
    I = CartesianIndex(i,j)
    reconstructedSource[I] = WaterLily.mult(I,μ₀,pp.D,realSol)
end
optPois=="Pois" && (pTest = WaterLily.Poisson(solIni, μ₀, reconstructedSource));
optPois=="MulP" && (pTest = WaterLily.MultiLevelPoisson(solIni, μ₀, source));
# reconstructedSource.*= (r .> R);

false

In [8]:
stepbystep && for i ∈ 1:200
    partRes = WaterLily.solver!(pTest;log=true,tol=1e-12,itmx=20)
    # pTest.x .-= mean(pTest.x[2:end-1,2:end-1]) 
    pythonplot()
    contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],pTest.x[2:end-1,2:end-1];levels=32)
    savefig("bunchFig/" * computationID * "Calculated_"*string(i, pad=4)*".png")
    # pythonplot()
    # contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(realSol[2:end-1,2:end-1]-pTest.x[2:end-1,2:end-1]).+1e-12);levels=32)
    # savefig("bunchFig/" * computationID * "Error_"*string(i, pad=4)*".png")

    push!(res,partRes...)
end

!stepbystep && (res = WaterLily.solver!(pTest;log=true,tol=1e-12,itmx=5e3));

In [9]:
#print(res)
r = sqrt.((X.-xc).^2+(Y.-yc).^2);
pTest.x .-= mean(pTest.x[2,2:end-1]);
realSol .-= mean(realSol[2,2:end-1]);
#pTest.x .*= (r .> R);
mu = (μ₀[2:end-1,2:end-1,1]+μ₀[3:end,2:end-1,1]+μ₀[2:end-1,2:end-1,2]+μ₀[2:end-1,3:end,2])/4
error = realSol[2:end-1,2:end-1]-pTest.x[2:end-1,2:end-1]

640×128 Matrix{Float64}:
 -2.22108e-6   -2.17296e-6   -2.01019e-6   …  -2.11476e-6   -2.27922e-6
 -2.60147e-6   -2.43703e-6   -2.3906e-6       -2.49517e-6   -2.54327e-6
 -3.2957e-6    -3.24761e-6   -3.08487e-6      -3.18941e-6   -3.35384e-6
 -4.53635e-6   -4.37195e-6   -4.32558e-6      -4.43009e-6   -4.47815e-6
 -6.09063e-6   -6.04262e-6   -5.87994e-6      -5.98441e-6   -6.14877e-6
 -8.19105e-6   -8.02674e-6   -7.98047e-6   …  -8.08489e-6   -8.13284e-6
 -1.06047e-5   -1.05568e-5   -1.03942e-5      -1.04986e-5   -1.06629e-5
 -1.35641e-5   -1.33999e-5   -1.33538e-5      -1.3458e-5    -1.35058e-5
 -1.68361e-5   -1.67884e-5   -1.66259e-5      -1.67301e-5   -1.68942e-5
 -2.06532e-5   -2.04891e-5   -2.04432e-5      -2.05473e-5   -2.05949e-5
  ⋮                                        ⋱                
 -0.000428449  -0.000428262  -0.000428243     -0.000428345  -0.000428366
 -0.000425092  -0.000425071  -0.000424886     -0.000424988  -0.000425176
 -0.000422324  -0.000422136  -0.000422117     -0

In [22]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],originalSource[2:end-1,2:end-1];levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "Source.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Cylinder_Neu_Source.png"

In [11]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],realSol[2:end-1,2:end-1];levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "Correct.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Cylinder_Neu_Correct.png"

In [12]:
pythonplot()
plot(res)
plot!(yscale=:log10, minorgrid=true)
savefig(computationID * "Residual.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Cylinder_Neu_Residual.png"

In [13]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],pTest.x[2:end-1,2:end-1];levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "Calculated.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Cylinder_Neu_Calculated.png"

In [14]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(error).+1e-12);levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "Error.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Cylinder_Neu_Error.png"

In [21]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(reconstructedSource[2:end-1,2:end-1]));levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "ReconstructedSource.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Cylinder_Neu_ReconstructedSource.png"

In [16]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(originalSource[2:end-1,2:end-1]-reconstructedSource[2:end-1,2:end-1]));levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "DiffSource.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Cylinder_Neu_DiffSource.png"

In [17]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],log10.(abs.(reconstructedSource[2:end-1,2:end-1]./(originalSource[2:end-1,2:end-1].+1e-12)));levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "SourceRatio.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Cylinder_Neu_SourceRatio.png"

In [18]:
pythonplot()
#plot(pTest.x[:,Int(N[2]/2)])
#plot(reconstructedSource[:,Int(N[2]/2)]+reconstructedSource[:,Int(N[2]/2+1)])
#plot(μ₀[:,Int(N[2]/2),2])
plot(error[:,Int(N[2]/2)])
plot!(minorgrid=true)
savefig(computationID * "CenterP.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Cylinder_Neu_CenterP.png"

In [19]:
pythonplot()
contourf(X[2:end-1,2:end-1],Y[2:end-1,2:end-1],mu;levels=32,aspect_ratio = :equal)
plot!(size=figureSize)
savefig(computationID * "Kernel.png")

"/home/tyhuang/Documents/Github/WaterLily.jl/test/TYTest/Pois_Cylinder_Neu_Kernel.png"

In [20]:
μ₀[Int(Ng[1]/2-R):Int(Ng[1]/2+R),Int(Ng[2]/2-R):Int(Ng[2]/2+R),2]

65×65 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  …  1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 ⋮                        ⋮              ⋱            ⋮                   
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0     